In [ ]:
#!import "../1-models/Azure.ipynb"
#!import "../3-operations/Connection.ipynb"

In [ ]:
var azureCreator = configuration.ContainsKey("azureCreator") ?
    configuration["azureCreator"] :
    null;

var environmentIdentifier = configuration.ContainsKey("environmentIdentifier") ?
    configuration["environmentIdentifier"] :
    "test";

var azureEnvironment = new AzureEnvironment(
    new User(azureCreator),
    environmentIdentifier
);

# Subscriptions

This operational notebook helps an administrator to manage subscriptions within the Azure Marketplace.

In [ ]:
var subscriptionsInEnvironment = Given<AzureEnvironment>.Match((environment, facts) =>
    from subscription in facts.OfType<Subscription>()
    where subscription.environment == environment
    select new
    {
        subscription,
        userIdentities = facts.OfType<SubscriptionUserIdentity>().Where(sui => sui.subscription == subscription),
        plans = facts.OfType<SubscriptionPlan>().Where(sp => sp.subscription == subscription),
        activations = facts.OfType<Activate>().Where(a => a.subscription == subscription),
        deactivations = facts.OfType<Deactivate>().Where(d => d.activate.subscription == subscription),
        suspensions = facts.OfType<Suspend>().Where(s => s.subscription == subscription),
        reinstatements = facts.OfType<Reinstate>().Where(r => r.suspend.subscription == subscription),
        renewals = facts.OfType<Renew>().Where(r => r.subscription == subscription)
    }
);

var subscriptions = await jinagaClient.Query(subscriptionsInEnvironment, azureEnvironment);

jinagaClient.RenderFacts(subscriptions)